In [76]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
import nltk
import numpy as np
from sklearn import preprocessing
import math

In [77]:
#subject6 的最明顯波長是68.4813s
#space = 17.12 #sample step size
#timeframe = 17.12
global q
q=0

In [78]:
class Subject():
    def __init__(self,subject):
        self.subject = subject

In [79]:
from tracemalloc import start


def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time
    
    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space

    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    global SBERT 
    SBERT= []

    def get_cosine_similarity(feature_vec_1, feature_vec_2):    
        return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]
    # Two lists of sentences
    for i in range(0,len(all_sentences001)-1):
        #Compute embedding for both lists
        embeddings1 = model.encode(all_sentences001[i], convert_to_tensor=True)
        #print(all_sentences001[i])
        embeddings2 = model.encode(all_sentences001[i+1], convert_to_tensor=True)
        #print(all_sentences001[i+1])
        #Compute cosine-similarities
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        #print(cosine_scores)
        # convert from PyTorch tensor to numpy array
        SBERT.append(get_cosine_similarity(embeddings1, embeddings2))
    ######
        print("timeframe: ",i,":",i+10,"\n")
        print(get_cosine_similarity(embeddings1, embeddings2),"\n")
        print(all_sentences001[i],"\n")
        print(all_sentences001[i+1],"\n")
        print()

    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global ancestor_depth_list
    ancestor_depth_list = []
    global ancestors_depth
    ancestors_depth = []
    global pair_counts
    pair_counts = []
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    Timeframe_percentage = []  #每個timeframe的valid pair percentage的list
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        validPairs = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        localMin_depth = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                
                hasAncestors = True
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):  # 標定第一個synset
                        for u in range(0, syns2_num): #標定第二個synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            wu_pulmer_sum += syns1[i].wup_similarity(syns2[u])
                            timeframe_pairs_count += 1
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                #localAncestors[sentence_num].append(syns1[i].lowest_common_hypernyms(syns2[u])[0])
                                localMin_depth+=(syns1[i].lowest_common_hypernyms(syns2[u])[0].min_depth())
                                hasAncestors = True
                else:
                    continue
        #這邊是跑完一句話的處理
        if hasAncestors:
            validPairs += 1
        if timeframe_pairs_count != 0:
            all_sentences_wu_sum_divided.append(wu_pulmer_sum/timeframe_pairs_count) #算出除以pairs數量的平均值
        else: 
            all_sentences_wu_sum_divided.append(wu_pulmer_sum)
        all_sentences_wu_sum.append(wu_pulmer_sum)
        pair_counts.append(timeframe_pairs_count)
        #print(all_sentences_wu_sum)
        global wu_difference
        wu_difference = []
        for x in range(len(all_sentences_wu_sum)-1):
            wu_difference.append(all_sentences_wu_sum[x+1]-all_sentences_wu_sum[x])
    # 計算ancestors' depth
        ancestor_depth_list.append(localMin_depth/timeframe_pairs_count)
    '''for i in range(len(localAncestors)):
        localMin_depth = 0
        for u in range(len(localAncestors[i])):
            localMin_depth += localAncestors[i][u].min_depth() #一句話所有min depth加起來
        ancestors_depth.append((localMin_depth/len(localAncestors[i])))'''
        #Timeframe_percentage += validPairs/(len(the_sentence)*(len(the_sentence)-1))
        #print(Timeframe_percentage)
    results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts,ancestor_depth_list)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts","ancestor_depth"])
    
    if q<10:
        results_csv.to_csv(f"data/Text/00{q}({destination}).csv")
    else:
        results_csv.to_csv(f"data/Text/0{q}({destination}).csv")
    print(timeframe_pairs_count)
    
    """ # for normalization
    wu_difference = np.array(wu_difference)
    SBERT = np.array(SBERT)
    #normalized_SBERT = preprocessing.normalize([SBERT])
    #normalized_wu_difference = preprocessing.normalize([wu_difference])


    min_max_scaler = preprocessing.MinMaxScaler()
    SBERT = np.reshape(SBERT,(479,1))
    wu_difference = np.reshape(wu_difference,(479,1))
    global normalized_SBERT
    global normalized_wu_difference
    normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
    normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)
    """


### 算common ancestor depth

In [80]:
def common_depth_calculate(gcs,timeframe,space,start_time=0,end=480): #所以gcs要是string


    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time

    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space


    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''   
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global actual_pair_counts
    actual_pair_counts = 0
    stops = set(stopwords.words('english'))
    global all_sentences_wu_sum
    all_sentences_wu_sum = []
    global all_sentences_wu_sum_divided
    all_sentences_wu_sum_divided = []
    global common_ancestors 
    common_ancestors = []
    global word_pair_counts
    word_pair_counts = 0
    global common_ancestor_synset_average_depth 
    common_ancestor_synset_average_depth  = 0
    global ancestor_depth_list
    ancestor_depth_list = []
    valid_word_pair_counts = 0 #計算有common ancestors的word pair
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字的
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                valid_pair_indicator = False
                word_pair_counts += 1
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num): #標定第一個字的synset
                        for u in range(0, syns2_num): #標定第二個字的synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            #print(syns1[i],"+",syns2[u])
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                common_ancestors.append(syns1[i].lowest_common_hypernyms(syns2[u])) 
                                ancestor_depth_list += [syns1[i].lowest_common_hypernyms(syns2[u])[0].min_depth()]
                                actual_pair_counts += 1
                                valid_pair_indicator = True
                            else:
                                ancestor_depth_list += [1] #把毫不相干的兩個字當作建立在depth = 1的ancestor上面
                                actual_pair_counts += 1
                                #print(syns1[i].lowest_common_hypernyms(syns2[u]))
                            common_ancestor_synset_average_depth += ancestor_depth_list[i*syns2_num+u]/(syns1_num*syns2_num)#這邊還在修改中，先算出任一synset ancestor depth/兩個字總synset對數
                    if valid_pair_indicator == True:
                        valid_word_pair_counts += 1
                else:
                    continue
        
    
        #print(all_sentences_wu_sum)
    
    common_ancestors_depth = 0
    for i in range(len(common_ancestors)):
        common_ancestors_depth += common_ancestors[i][0].min_depth() 
    #common_ancestors_depth = sum([x.min_depth() for x in common_ancestors]) #計算所有ancestor 的深度總和
    average_ancestors_depth = common_ancestors_depth/len(common_ancestors) #計算common ancestors' depth 的平均
    print("average_ancestors_depth=",average_ancestors_depth)
    print("ancestor_depth_averaged_by_word_counts(none as 1)=",common_ancestor_synset_average_depth/word_pair_counts )
    print("synset pairs with ancestors=", len(common_ancestors),", actual synset pairs=", actual_pair_counts, ", percentage_of_ancestors", len(common_ancestors)/actual_pair_counts
            ,", word pairs=",word_pair_counts," ,valid_wp_percentage=",valid_word_pair_counts/word_pair_counts,"\n")

In [81]:
wordnet.synsets("dog")[0].min_depth()


8

def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string

In [82]:
#計算單一SUBJECT的切割
for i in range(1,25):
    try:
        x=i
        q=x
        calculate(f"data/Text/{q:0>3d}(done).csv",f"ancestor",10,1)
    except:
        continue

timeframe:  0 : 10 

0.94828385 

hello my name is Angelinis and  I am 21 years old I was born in Bogota Colombia and in the place I used 

my name is Angelinis and  I am 21 years old I was born in Bogota Colombia and in the place I used to live 


timeframe:  1 : 11 

0.8144996 

my name is Angelinis and  I am 21 years old I was born in Bogota Colombia and in the place I used to live 

and  I am 21 years old I was born in Bogota Colombia and in the place I used to live with my mum 


timeframe:  2 : 12 

0.9860154 

and  I am 21 years old I was born in Bogota Colombia and in the place I used to live with my mum 

I am 21 years old I was born in Bogota Colombia and in the place I used to live with my mum 


timeframe:  3 : 13 

0.88233733 

I am 21 years old I was born in Bogota Colombia and in the place I used to live with my mum 

years old I was born in Bogota Colombia and in the place I used to live with my mum and my brother 


timeframe:  4 : 14 

0.9117321 

years old I was born

In [83]:
x=8
startt=0
endt=100
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)

x=8
startt=100
endt=260
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)

x=8
startt=300
endt=480
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)

x=8
startt=100
endt=300
common_depth_calculate(f"data/Text/{x:0>3d}(done).csv",endt-startt,4880,start_time=startt,end=endt)

average_ancestors_depth= 0.9620787488755104
ancestor_depth_averaged_by_word_counts(none as 1)= 0.8584893320533801
synset pairs with ancestors= 86706 , actual synset pairs= 1049513 , percentage_of_ancestors 0.08261546069462694 , word pairs= 8325  ,valid_wp_percentage= 0.501981981981982 

average_ancestors_depth= 0.973187169467696
ancestor_depth_averaged_by_word_counts(none as 1)= 0.905584115268728
synset pairs with ancestors= 199867 , actual synset pairs= 2294529 , percentage_of_ancestors 0.08710589406366187 , word pairs= 24002  ,valid_wp_percentage= 0.5292058995083743 

average_ancestors_depth= 0.8972702465552652
ancestor_depth_averaged_by_word_counts(none as 1)= 0.9264553061273707
synset pairs with ancestors= 311127 , actual synset pairs= 3487580 , percentage_of_ancestors 0.08920999661656506 , word pairs= 31885  ,valid_wp_percentage= 0.524290418692175 

average_ancestors_depth= 0.9625260092778705
ancestor_depth_averaged_by_word_counts(none as 1)= 0.8960690752552433
synset pairs with a

0.9092529740477905

參數表:<br>
all_sentences001: 所有的句子整合(沒有刪掉stop words)<br>
SBERT: 具有全部SBERT similarity的list<br>
all_sentences_wu_sum: 每句話的wu-pulmer similarity算出來的平均的list<br>
wu_difference: 兩句兩句減掉wu similarity的list<br>
normalized_SBERT: 就是表準化過後的SBERT list<br>
normalized_wu_difference: 標準化過後的wu_pulmer difference<br>
這邊注意wu-pulmer的different based應該不能這樣算

一般SD切割
halfpeakstep_phasef

timestep=1 SD切割
wholehalff


In [84]:
'''
#計算單一SUBJECT的切割
x=10
q=x
freq = float(input("enter freq: "))
period = 1/(freq/1000)
startt = input("enter start time: ")
endt= input("enter end time:")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}",period/2,1,cut_timeoffset,cut_timeend)
'''

'\n#計算單一SUBJECT的切割\nx=10\nq=x\nfreq = float(input("enter freq: "))\nperiod = 1/(freq/1000)\nstartt = input("enter start time: ")\nendt= input("enter end time:")\ncut_timeoffset = int(startt)\ncut_timeend = int(endt)\ncalculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}",period/2,1,cut_timeoffset,cut_timeend)\n'

抓週期

In [85]:
def read_period(gcs): #所以gcs要是string
    df = pd.read_csv(gcs) 
    global period
    period = df["period(Wu)"][20] #現在是抓wu-pulmer divided 的週期
    #print(period)
    

一般切割SD

In [86]:
#這個切半個週期SD用的
x=8
q=x
freq = float(input("enter freq: "))
period = 1/(freq/1000)
startt = input("enter start time: ")
endt= input("enter end time:")
#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
phasenum=0
for t in range(cut_timeoffset,cut_timeoffset+int(period/2)): #這邊驅動的是不同phase的時間差
    print("start_time =", t)
    phasenum+=1
    calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phasef{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
print("phase有:",phasenum)
print("freq=", freq)
print(startt,endt)


ValueError: could not convert string to float: ''

gcs,destination,timeframe,space,start_time=0,end=480

In [ ]:
#這個好像是切整套SD用的
x=13
q=x
freq = float(input("enter freq: "))
period = 1/(freq/1000)
startt = input("enter start time: ")
endt= input("enter end time:")
#read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
phasenum=0
for t in range(cut_timeoffset,cut_timeend-int(period/2)): #這邊驅動的是不同phase的時間差
    print("start_time =", t)
    phasenum+=1
    calculate(f"data/Text/0{x}(done).csv",f"wholehalff{math.floor(freq)}_{startt}_{endt}_{t-cut_timeoffset}",period/2,period/2,t,end = cut_timeend) #,
print("phase有:",phasenum)
print("freq=", freq)



ValueError: could not convert string to float: ''

In [ ]:
'''
start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            for t in range(0,int(period/2)):
                print("start_time =", t)
                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)
        except:
            print("error:",x)
            continue
'''

'\nstart_subject = int(input("start_subject = "))\nend_subject = int(input("end_subject = "))\nfor x in range(start_subject,end_subject+1):\n    if x<10:\n        try: \n            q=x\n            read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/00{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n        \n    else:\n        q=x\n        try:\n            read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")\n            for t in range(0,int(period/2)):\n                print("start_time =", t)\n                calculate(f"data/Text/0{x}(done).csv",f"halfpeakstep_phase0f{t}",period/2,period/2,t)\n        except:\n            print("error:",x)\n            continue\n'

In [ ]:

start_subject = int(input("start_subject = "))
end_subject = int(input("end_subject = "))
for x in range(start_subject,end_subject+1):
    if x<10:
        try: 
            q=x
            #read_period(f"data/peaks_of_PSD/00{x}(peaks).csv")
            calculate(f"data/Text/00{x}(done).csv","step0303",3,3)
        except:
            print("error:",x)
            continue
        
    else:
        q=x
        try:
           #read_period(f"data/peaks_of_PSD/0{x}(peaks).csv")
            calculate(f"data/Text/0{x}(done).csv","step0303",3,3)
        except:
            print("error:",x)
            continue


ValueError: invalid literal for int() with base 10: ''

In [ ]:
print(len(all_sentences001))
print(len(SBERT))

160
159


In [ ]:
'''
for i in range(18,24):
    if i<10:
        try:
            calculate(f"data/Text/00{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/00{i}(results).csv')
        except:
            print("error:", i)
            continue
    else:
        try:
            calculate(f"data/Text/0{i}(done).csv")
            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = ['SBERT',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])
            results_csv.to_csv(f'data/Text/0{i}(results).csv')
        except:
            print("error:", i)
            continue
'''

'\nfor i in range(18,24):\n    if i<10:\n        try:\n            calculate(f"data/Text/00{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/00{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n    else:\n        try:\n            calculate(f"data/Text/0{i}(done).csv")\n            results_csv = pd.DataFrame(list(zip(SBERT,all_sentences_wu_sum,all_sentences_wu_sum_divided,pair_counts)), columns = [\'SBERT\',"Wu-Pulmer","Wu-Pulmer_divided","pair_counts"])\n            results_csv.to_csv(f\'data/Text/0{i}(results).csv\')\n        except:\n            print("error:", i)\n            continue\n'